In [37]:
!pip install jsonlines
!pip install python-docx
!pip install docx2pdf

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
import jsonlines
from tqdm import tqdm
import re, os

from docx import Document
from docx.shared import Inches
from docx2pdf import convert

In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz

--2021-11-19 14:43:04--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4131 (4.0K) [application/x-gzip]
Saving to: ‘msmarco-test2020-queries.tsv.gz.1’

msmarco-test2020-qu 100%[===================>]   4.03K  --.-KB/s    in 0s      

2021-11-19 14:43:04 (524 MB/s) - ‘msmarco-test2020-queries.tsv.gz.1’ saved [4131/4131]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!gunzip /content/msmarco-test2020-queries.tsv.gz

In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco_v2_doc.tar

--2021-11-19 13:37:10--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco_v2_doc.tar
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34648862720 (32G) [application/x-tar]
Saving to: ‘msmarco_v2_doc.tar’

msmarco_v2_doc.tar  100%[===================>]  32.27G  45.4MB/s    in 18m 50s 

2021-11-19 13:55:59 (29.3 MB/s) - ‘msmarco_v2_doc.tar’ saved [34648862720/34648862720]



In [ ]:
!ls
%cd ../..
!ls
%cd content
!ls

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr
/
bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr
/content
msmarco-test2020-queries.tsv	   msmarco_v2_doc
msmarco-test2020-queries.tsv.gz    msmarco_v2_doc.tar
msmarco-test2020-queries.tsv.gz.1  sample_data


In [ ]:
#/content/sample_data
!tar -xvf "/content/msmarco_v2_doc.tar" -C "/content/msmarco_v2_doc/"

tar: /content/msmarco_v2_doc: Cannot open: Not a directory
tar: Error is not recoverable: exiting now


In [ ]:
!mkdir data
%cd data
!mkdir robust04
%cd robust04

!wget https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/topics.robust04.txt
!wget https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/qrels.robust04.txt
!wget https://storage.googleapis.com/castorini/robust04/trec_disks_4_and_5_concat.txt
%cd ../../

/content/data
/content/data/robust04
--2021-12-07 11:13:12--  https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/topics.robust04.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116975 (114K) [text/plain]
Saving to: ‘topics.robust04.txt’

topics.robust04.txt 100%[===================>] 114.23K  --.-KB/s    in 0.02s   

2021-12-07 11:13:12 (5.16 MB/s) - ‘topics.robust04.txt’ saved [116975/116975]

--2021-12-07 11:13:12--  https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/qrels.robust04.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [ ]:
def load_corpus(path, html=False):
    print('Loading corpus...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = f.read()
        for raw_text in tqdm(all_text.split('</DOC>')):
            if not raw_text:
                continue
            result = re.search(r'\<DOCNO\>(.*)\<\/DOCNO\>\n', raw_text)
            if not result:
                continue
            doc_id = result.group(1)
            
            doc_id = doc_id.replace('0','A').replace('1','B').replace('2','C').replace('3','D').replace('4','E')
            doc_id = doc_id.replace('5','F').replace('6','G').replace('7','H').replace('8','I').replace('9','J')
            doc_id = doc_id.replace('-','M')
            
            doc_id = doc_id.strip()
            raw_text = ' '.join(raw_text.split())

            result = re.search(r'\<TEXT\>(.*)\<\/TEXT\>', raw_text)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1
                    
            corpus[doc_id] = ' '.join(doc_text.split())
            
            # break

    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

In [ ]:
corpus = load_corpus('/content/data/robust04/trec_disks_4_and_5_concat.txt')

Loading corpus...


100%|██████████| 528160/528160 [00:56<00:00, 9319.69it/s]

Loaded 528155 docs, 523877 with texts.


In [ ]:
corpus['FT931-16655']

"REDLAND, the building materials group which paid Pounds 624m in shares for Steetley last year, has raised Pounds 100m through disposals. It is reinvesting Pounds 58.6m in its continental European tile and brick activities. Redland is forming a joint venture with Koramic, a private Belgian company, encompassing the two companies' brick making in the Netherlands, Germany and Belgium. The new company will be the largest facing brick maker in continental Europe. Profits of the combined activities in 1992 would have been Pounds 15.9m, of which Redland's share would have been Pounds 5.8m. The 50:50 joint venture will own 70 per cent of the combined activities, giving Redland a 35 per cent economic interest but joint control. Redland will also receive Pounds 17.3m in cash. Redland is increasing its stake in Coverland, the French roof tile producer, from 43 per cent to 67 per cent by acquiring part of St Gobain's shareholding. Options for Redland to purchase St Gobain's remaining 33 per cent 

In [ ]:
!ls
!mkdir html
%cd html

/content/data/html/html


In [ ]:
for i, key in enumerate(corpus.keys()):

    with open('/content/data/htmlABC/' + key + '.html','w') as writer:  

        writer.write("<!DOCTYPE html>\n")
        writer.write("<html lang=\"en-US\">\n")
        writer.write("\t<head>\n")
        writer.write("\t\t<meta charset=\"UTF-8\">\n")
        writer.write("\t\t<title>" + key + "</title>\n")
        writer.write("\t</head>\n")
        writer.write("\t<body>\n")
        writer.write("\t\t<main>\n\n")
        writer.write("\t\t\t<p>" + corpus[key] + "</p>\n\n")
        writer.write("\t\t</main>\n")
        writer.write("\t</body>\n")
        writer.write("</html>")
        

    if i == 99:
        break

In [ ]:
with open('/content/data/sitemap.xml','w') as writer:    
  
    writer.write('<?xml version="1.0" encoding="UTF-8"?>\n\n')
    writer.write('<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n\n')

    for key, values in corpus.items():
        writer.write('\t<url>\n')
        writer.write('\t\t<loc>https://storage.googleapis.com/robust04-public/' + str(key) + '.pdf</loc>\n')
        writer.write('\t\t<lastmod>2021-12-7</lastmod>\n')
        writer.write('\t</url>\n\n')

    for key, values in corpus.items():
        writer.write('\t<url>\n')
        writer.write('\t\t<loc>https://storage.googleapis.com/robust04-public/' + str(key) + '.html</loc>\n')
        writer.write('\t\t<lastmod>2021-12-7</lastmod>\n')
        writer.write('\t</url>\n\n')
        
    writer.write('\t</urlset>\n\n')

In [ ]:
with open('/content/data/sitemap.txt','w') as writer:    
  
    writer.write('https://storage.googleapis.com/robust04-public/doc1.txt\n')
    writer.write('https://storage.googleapis.com/robust04-public/doc2.txt\n')
    writer.write('https://storage.googleapis.com/robust04-public/Plano%20De%20Atividades%20-%20Bolsistas.pdf\n')
    writer.write('https://storage.googleapis.com/robust04-public/doc3.html\n')
    writer.write('https://storage.googleapis.com/robust04-public/doc4.html\n')
    writer.write('https://storage.googleapis.com/robust04-public/cars.pdf\n') 

    for key, values in corpus.items():
        writer.write('https://storage.googleapis.com/robust04-public/' + str(key) + '.pdf\n')

In [ ]:
i = 0
j = 1

path_d = '/content/data/docx/'
os.mkdir(path_d)

path_p = '/content/data/pdf/'
os.mkdir(path_p)

FileExistsError: ignored

In [ ]:
for key, values in corpus.items():

    document = Document()
    p = document.add_paragraph(values)
    document.save(path_d + str(key) + '.docx')

    convert(path_d + str(key) + '.docx', path_p + str(key) + '.pdf')

NotImplementedError: ignored

In [ ]:
!pip install comtypes

     |████████████████████████████████| 145 kB 14.0 MB/s 
  Created wheel for comtypes: filename=comtypes-1.1.10-py3-none-any.whl size=164937 sha256=7f7bc96b0a4c4884e4c59db9a255c0b9db4e0c567e8b966ea2984afe40181146
  Stored in directory: /root/.cache/pip/wheels/ac/d9/86/f150377841ceef17cc9831ead2989c241b39e973269b347653
Successfully built comtypes


In [ ]:
import sys
import os
import comtypes.client

os. chdir(r'setting current WD')

wdFormatPDF = 17

in_file = r"/content/data/docx/FT931-16655.docx"
out_file = r"/content/data/pdf/FT931-16655.pdf"

word = comtypes.client.CreateObject('Word.Application')
doc = word.Documents.Open(in_file)
doc.SaveAs(out_file, FileFormat=wdFormatPDF)
doc.Close()
word.Quit()
print('File Converted!')

ImportError: ignored

In [ ]:
i = 0
j = 1
path = '/content/data/PDF/pdf/'
os.mkdir(path)


In [ ]:
i = 0
j = 1
path = '/content/data/PDF/pasta_1/'
os.mkdir(path)
for key, values in corpus.items():
    
    i += 1

    pdf = FPDF()  

    pdf.accept_page_break()
    pdf.set_margins(left = 5, top = 5)
    pdf.add_page()
    pdf.set_font("Arial", size = 12)
    
    pdf.multi_cell(200, 10, txt = str(values), align = 'J')
    pdf.output(path + str(key) + '.pdf')   
    
    if i == 500:
        j += 1
        pasta = 'pasta_' + str(j)
        path = '/content/data/PDF/' + pasta + '/'
        os.mkdir(path)
        
        i = 0

NameError: ignored

In [ ]:
!gsutil version -l

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
gsutil version: 5.5
checksum: 54a50a74649e27e7568328e5852187f0 (OK)
boto version: 2.49.0
python version: 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
OS: Linux 5.4.104+
multiprocessing available: True
using cloud sdk: True
pass cloud sdk credentials to gsutil: True
config path(s): No config found
gsutil path: /tools/google-cloud-sdk/bin/gsutil
compiled crcmod: True
installed via package manager: False
editable install: False


In [ ]:
!gcloud auth login
!gcloud auth application-default login

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=0Lxumllx134BvxNBMa0FI9jcUJU2d8&prompt=consent&access_type=offline&code_challenge=L1A562fdKbDuukzRzbMxfkPmjOydgOQMATdptZ9DVnI&code_challenge_method=S256

Enter verification code: 4/1AX4XfWiJ-20Iu0gy57tlP59xCwU-7HTkPGgmJJLKqxZNpnTvAPPpI4AgoOw

You are now logged in as [g182786@g.u

In [ ]:
!gsutil cp /content/data/PDF/pasta_3/* gs://robust04-public/

Copying file:///content/data/PDF/pasta_3/FT931-13842.pdf [Content-Type=application/pdf]...
Copying file:///content/data/PDF/pasta_3/FT931-13843.pdf [Content-Type=application/pdf]...
Copying file:///content/data/PDF/pasta_3/FT931-13844.pdf [Content-Type=application/pdf]...
Copying file:///content/data/PDF/pasta_3/FT931-13845.pdf [Content-Type=application/pdf]...
- [4 files][  7.0 KiB/  7.0 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/data/PDF/pasta_3/FT931-13846.pdf [Content-Type=application/pdf]...
Copying file:///content/data/PDF/pasta_3/FT931-13847.pdf [Content-Type=application/pdf]...
Copying file:///content/data/PDF/pasta_3/FT931-13848.pdf [Content-Type=application/pdf]...
Copying file:///content/data/P

In [ ]:
!gsutil -m cp /content/data/htmlABC* gs://robust04-public/

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
Omitting directory "file:///content/data/htmlABC". (Did you mean to do cp -r?)
CommandException: No URLs matched. Do the files you're operating on exist?
